how can we select some information for those auctions which had their final_price higher than the average final_price? We have to use a subquery. Take a look:
```
SELECT
  id,
  final_price 
FROM (
  SELECT
    id,
    final_price,
    AVG(final_price) OVER() AS avg_final_price
  FROM auction) c
WHERE final_price > avg_final_price
```
In the FROM clause, we introduced a subquery where we selected both the final_price for each auction and the average final_price. Because the whole subquery is calculated before the external query, we can use avg_final_price in the external query.

Find the id, country and views for those auctions where the number of views was below the average.
```
select
	id, country, views
from (select id, country, views, avg(views) over() as avg_views
      from auction) d
where views < avg_views;
```

Just as we expected, no window functions are allowed in HAVING either. Okay, you know that the remedy is to use a subquery. Try to correct the query on your own. Don't worry if you can't, the hint will be waiting for you in case you need it.

Exercise
Again, we would like to show those countries (country name and average final price) that have the average final price higher than the average price from all over the world. Correct the query by using a subquery.

```
SELECT 
  country, 
  AVG(final_price) 
FROM auction 
GROUP BY country 
HAVING AVG(final_price) > (select avg(final_price) from auction);
```

```
SELECT
  quartile,
  MIN(views),
  MAX(views)
FROM
  (SELECT
    views,
    ntile(4) OVER(ORDER BY views DESC) AS quartile
  FROM auction) c
GROUP BY quartile;
```
We used the window function in the inner query, which is why we could use it for grouping in the external query.

Now, divide all auctions into 6 equal groups based on the asking_price in ascending order. Show columns group_no, minimal, average and maximal value for that group. Sort by the group in ascending order.
```
select
	x.group_no,
    MIN(x.asking_price),
    AVG(x.asking_price),
    MAX(x.asking_price)
from (select
          asking_price,
          ntile(6) over(order by asking_price) as group_no
      from auction) x
group by x.group_no
```

So, to sum up this section, remember the following rule: the only places where we can use window functions without having to write subqueries are the SELECT and ORDER BY clauses. In all other places you have to use subqueries.

Exercise
You've seen several examples where the window function was placed in the SELECT clause, but how about trying to put it in the ORDER BY clause now?

For each auction, show the following columns: id, views and quartile based on the number of views in descending order. Order the rows by the quartile.

```
select
	id, views, 
    ntile(4) over(order by views desc) as quartile
from auction
order by ntile(4) over(order by views desc)
;
```

Before, we said that window functions were calculated after the GROUP BY clause. This has a very important implication for our queries: if the query uses any aggregates, GROUP BY or HAVING, the window function sees the group rows instead of the original table rows.

To get a better understanding of this phenomenon, take a look at the following example:
```
SELECT 
  category_id,
  final_price, 
  AVG(final_price) OVER() 
FROM auction;
```
This simple query will show the id and final_price of each auction alongside the average final_price from all the auctions. Now, take a look at the modified example with grouping:
```
SELECT 
  category_id,
  MAX(final_price), 
  AVG(final_price) OVER() 
FROM auction 
GROUP BY category_id;
```
Will this query work? Let's find out!

However, let's take a look at another modification of this example:
```
SELECT
  category_id,
  MAX(final_price) AS max_final, 
  AVG(MAX(final_price)) OVER()
FROM auction
GROUP BY category_id;
```
What will happen now?

Exercise
Run the template.

As you can see, the query now succeeded because we used an aggregate function (MAX(final_price)) that was indeed available after grouping the rows. By the way, this is the only place where you can nest aggregate functions inside one another.

Okay. The best way to correctly create queries with window functions and GROUP BY is as follows: first, create the query with GROUP BY, but without window functions. Run the query (in the database or in your head). Now, the columns you see in the result are the only columns you can use in your window functions.

Exercise
Group the auctions by the country. Show the country, the minimal number of participants in an auction and the average minimal number of participants across all countries.

```
select
	country,
    MIN(participants),
    AVG(MIN(participants)) over()
from auction
group by country;
```

Group the auctions by category_id and show the category_id and maximal asking price in that category alongside the average maximal price across all categories.
```
select
	category_id,
    MAX(asking_price),
    AVG(MAX(asking_price)) over()
from auction
group by category_id
```

As you can see, it's fairly simple to create quite advanced statistics very easily thanks to how window functions behave with GROUP BY. Let's take a look at other use cases.

For instance, we may make a ranking based on an aggregate function. Take a look:
```
SELECT
  country,
  COUNT(id),
  RANK() OVER(ORDER BY COUNT(id) DESC)
FROM auction
GROUP BY country;
```
We grouped auctions with respect to the country, counted the number of auctions from each country... and then we created a ranking based on that count of auctions.

Exercise
Now, group the auctions based on the category. Show category_id, the sum of final prices for auctions from this category and a ranking based on that sum, with the highest sum coming first.
```
select
	category_id,
    sum(final_price),
    rank() over(order by sum(final_price) desc)
from auction
group by category_id
```

Group the auctions based on the day they ended and show the following columns: ended, the average number of views from auctions on that day and the ranking based on that average (the highest average should get the rank of 1).

```
select
	ended,
    AVG(views),
    rank() over(order by avg(views) desc)
from auction
group by ended
```

Great. Another thing we can do with window functions when rows are grouped are leads, lags and day-to-day deltas. Take a look:
```
SELECT
  ended,
  SUM(final_price) AS sum_price,
  LAG(SUM(final_price)) OVER(ORDER BY ended)
FROM auction
GROUP BY ended
ORDER BY ended
```
The above query shows each end date with the total price of all items sold on that day and the same total price on the previous day.

Exercise
For each end day, show the following columns:
- ended.
- the sum of views from auctions that ended on that day.
- the sum of views from the previous day (name the column previous_day.
- delta – the difference between the sum of views on that day and on the previous day (name the column delta).

```
select
	ended,
    sum(views),
    lag(sum(views)) over(order by ended) as previous_day,
    sum(views) - lag(sum(views)) over(order by ended) as delta
from auction
group by ended
```

Perfect. Finally, you can use window functions with PARTITION BY on grouped rows. One thing you need to remember is that the window function will only see grouped rows, not the original rows. Take a look:
```
SELECT
  country,
  ended,
  SUM(views) AS views_on_day,
  SUM(SUM(views)) OVER(PARTITION BY country)
    AS views_country
FROM auction
GROUP BY country, ended
ORDER BY country, ended
```
The query might require a bit of explanation. First of all, we grouped all rows by the country and ended. Then, we showed the country name and date when the auctions ended. Look what happens in the next two columns. First, we simply sum the views in accordance with our GROUP BY clause, i.e. we get the sum of views in all auctions from the particular country on the particular day. But look what happens next. We use a window function to sum all daily sums for a particular country. As a result, we get the sum of views for a particular country on all days.

Exercise
Group all auctions by the category and end date and show the following columns:

- category_id.
- ended.
- the average daily final price as daily_avg_final_price in that category on that day.
- the maximal daily average in that category from any day as daily_max_avg.

```
select
	category_id,
    ended,
    AVG(final_price) as daily_avg_final_price,
    MAX(AVG(final_price)) over(partition by category_id) as daily_max_avg
from auction
group by category_id, ended
```

Divide the books into 4 groups based on their rating. For each group (bucket), show its number (column bucket), the minimal and maximal rating in that bucket.

```
select 
	bucket,
    MIN(rating), MAX(rating)
from (Select 
	id,
    rating,
    ntile(4) over(order by rating) as bucket
from book) x
group by bucket
order by bucket
```

For each author show:

- author_id.
- the number of books published by this author (name the column number_of_books).
- the rank of the author based on the number of published books in descending order.

```
select
	author_id,
    count(id) as number_of_books,
    rank() over(order by count(id) desc)
from book
group by author_id
```

For each year in which books were published, show the following columns:

publish_year.
the number of books published that year.
the number of books published in the previous year.

```
select
	publish_year,
    count(id),
    lag(count(id)) over(order by publish_year)
from book
group by publish_year
```